In [1]:
import os 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, log_loss 
import pickle

In [2]:
base_dir_mit = '/data/ddmg/scate/multiple_shortcut/waterbirds'
base_dir_gl = '/nfs/turbo/coe-soto/mmakar/multiple_shortcut/waterbirds'
if os.path.exists(base_dir_mit):
    base_dir = base_dir_mit
elif os.path.exists(base_dir_gl):
    base_dir = base_dir_gl 
pixel = 128
batch_size = 64

In [3]:
def get_per_dist_metrics(df, model_name):
    model_eval = {'hash': df.model.iloc[0]}
    if 'unweighted' in model_name:
        model_eval['validation_new_auc'] = roc_auc_score(
            df.y0, df.pred0)
        model_eval['validation_pred_log_loss'] = log_loss(
            df.y0, df.pred0)
    else: 
        model_eval['validation_new_auc'] = roc_auc_score(
            df.y0, df.pred0, sample_weight=df.sample_weights)
        model_eval['validation_pred_log_loss'] = log_loss(
            df.y0, df.pred0, sample_weight=df.sample_weights)
    return model_eval 

def update_performance(df, model_name): 
    old_eval = pickle.load(open(
        f"{base_dir}/tuning/{df['model'].iloc[0]}/performance.pkl", "rb"))
#     if 'validation_micro_auc' in old_eval.keys():
#         return old_eval 
    model_eval = get_per_dist_metrics(df, model_name)
    old_eval.update(model_eval)
    pickle.dump(old_eval, open(f"{base_dir}/tuning/{model_eval['hash']}/performance.pkl", 
                    'wb'))
    return model_eval
    
def get_all_dist_metrics_all_models(model_name, xv_mode, base_dir,  v_dim):
    all_eval = []
    curr_pred = pd.read_csv(
        (f'{base_dir}/final_models/all_valid_pred_{model_name}_{xv_mode}'
         f'_pix{pixel}_bs{batch_size}_vdim{v_dim}.csv'))
    for model in curr_pred.model.unique().tolist():
        model_pred = curr_pred[(curr_pred.model == model)]
        model_eval = update_performance(model_pred, model_name)
        all_eval.append(pd.DataFrame(model_eval, index=[0]))
    all_eval = pd.concat(all_eval, ignore_index=True)
    return all_eval
            

In [9]:
model_name = 'weighted_hsic'
xv_mode = 'two_step'
v_dim = 12
valid_eval = get_all_dist_metrics_all_models(model_name, xv_mode, base_dir=base_dir, v_dim=v_dim)

In [10]:
valid_eval.shape

(30, 3)

In [28]:
import pandas as pd
import numpy as np
experiment_directory = (f'/nfs/turbo/coe-soto/mmakar/multiple_shortcut/waterbirds/experiment_data/'
    f'rs0')
data = pd.read_csv(
    f'{experiment_directory}/test_0.5.txt')
data = data['0'].str.split(",", expand=True)
data.columns = ['bird_img', 'bird_seg', 'back_img', 'noise_img'] + \
    [f'y{i}' for i in range(data.shape[1]-4)]
for i in range(13):
    data[f'y{i}'] = data[f'y{i}'].astype(np.float32)

In [29]:
data.y2[(data.y0==1.0)].value_counts(normalize=True)

0.0    0.50303
1.0    0.49697
Name: y2, dtype: float64

In [30]:
data.y2[(data.y0==0.0)].value_counts(normalize=True)

1.0    0.504076
0.0    0.495924
Name: y2, dtype: float64

In [27]:
0.35 + 0.65

1.0